# Prepare trainset for BIOE format

In [2]:
import pandas as pd
from openpyxl import load_workbook
import numpy as np
from sklearn.model_selection import train_test_split
pd.set_option('display.max_rows', None)

## Import the data

In [3]:
dfall = pd.read_excel('data/2022年10月天猫新品商品标题-split-清洗后merge.xlsx', engine = 'openpyxl')
dfall= dfall[['spu_id','spu_name','keywords_without_obsolete_y']]
dfall.head()

,spu_id,spu_name,keywords_without_obsolete_y
0,100019125569,联想拯救者Y9000P 2022 16英寸游戏笔记本电脑(12代 i7-12700H 16G...,笔记本电脑
1,100038993607,联想拯救者Y9000P 2022 16英寸游戏笔记本电脑(12代 i7-12700H 16G...,笔记本电脑
2,100033822492,联想拯救者Y9000P 2022 16英寸游戏笔记本电脑(12代 i7-12700H 16G...,笔记本电脑
3,679040984188,【24期免息 学生赠充电宝】OPPO 一加 Ace Pro 5g新品学生OnePlus游戏智...,5g手机
4,654578360475,【立即抢购】小米/红米Redmi Note 11 5G 5000mAh大电量智能红米手机5千...,5g手机


In [4]:
print("The length of the smaple: " + str(len(dfall)))

dis_spu = dfall['spu_id'].astype(str).unique()
print("The number of the distinct spu_id: " + str(len(dis_spu)))

The length of the smaple: 2516
The number of the distinct spu_id: 2516


## Data Preprocessing

In [5]:
# data preprocessing
newwordsNER = dfall['keywords_without_obsolete_y'].astype(str)
newwordsNER = newwordsNER.unique()

# check for the na
newwordsNER = [x for x in newwordsNER if x == x]
check_na = sum(pd.isna(newwordsNER))
print("The number of NA in this dataset: " + str(check_na))

# split words through ","
newwordsNER = [item for items in newwordsNER for item in items.split(",")]



The number of NA in this dataset: 0


In [6]:
# 去掉商品名称里的空字段；
X_train = dfall['spu_name']
for i in range(len(X_train)):
    X_t = X_train[i].replace(' ', '')
    X_train[i] = X_t 

#y
y_words = dfall['keywords_without_obsolete_y']
y_train = y_words

#X_train, X_test, y_train, y_test = train_test_split(X, y_words, test_size=0.2, random_state=42)

#X_train = pd.Series(X_train)
#X_test = pd.Series(X_test)
#y_train=y_train.reset_index(drop=True)
#y_test=y_test.reset_index(drop=True)

#print('Train size: {}, test size: {}' .format(X_train.shape, X_test.shape))

/var/folders/7y/cwfy92wj6rq6_kc_6n81nb1m0000gp/T/ipykernel_80941/1122195371.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[i] = X_t


## Create 2D array function 

In [7]:
# 创建存储label的空集
def init_list_of_objects(size):
    list_of_objects = list()
    for i in range(0,size):
        list_of_objects.append( list() ) #different object reference each time
    return list_of_objects

labels = init_list_of_objects(len(X_train))

# 按照每一个spu_name的长度创建一个存储的空集label；
i=0
for X_t in X_train:
    #print(X_t)
    labels[i]= ['']*len(str(X_t)) #test=X_train[0]
    i=i+1
   

In [8]:
# 建立空list存储实体关键词   
chars = []
i=0
for X_t in X_train:
    X_t = str(X_t).encode("utf-8").decode("utf-8")
    splitted = str(y_train[i]).encode("utf-8").decode("utf-8").split(',') #看对应的关键词
    #print(splitted)
    ll = []
    for word in splitted: #每个实体关键词
        temp = list(word.encode("utf-8").decode("utf-8"))##分割成一个个字符
        # 检查第1500个商品
        if i==1500:
            print(X_t)
            print(y_train[i])
            print(temp)
        ll.append(temp)
        
    chars.append(ll)
    i=i+1

1米鲜花花束礼盒彩灯串灯烘焙蛋糕装饰插件led灯串闪灯铜线星星灯
蛋糕装饰
['蛋', '糕', '装', '饰']


In [9]:
chars[90]

[['一', '体', '机']]

In [10]:
# 找到spu_name最长的长度
max_len = []
for i in range(len(chars)):
    for j in range(len(chars[i])):
        length = len(chars[i][j])
        max_len.append(length)

print("The max length of this sample size is: " + str(max(max_len)))

The max length of this sample size is: 13


## Create BIOE label 

In [11]:
## create BIOE labels 

i = 0 
for X_t in X_train:
    X_t = str(X_t).encode("utf-8").decode("utf_8")
    if i == 90:
        print(X_t)
        print(chars[90])
    try:
        for a in range(len(X_t)):
            for b in range(len(chars[i])):
                token_size = len(chars[i][b])
                if token_size == 2:
                    if (X_t[a] == chars[i][b][0]) and (X_t[a+1] == char[i][b][1]):
                        labels[i][a] = 'B'
                        labels[i][a+1] = 'E'
                elif token_size >= 2:
                    if(X_t[a] == chars[i][b][0]) and (X_t[a+1] == chars[i][b][1]) and (X_t[a+2] == chars[i][b][2]) and (X_t[a+token_size-1]==chars[i][b][token_size-1]):
                        labels[i][a] = 'B'
                        for c in range(token_size):
                            if c <= token_size-3:
                                labels[i][a+c+1] = 'I'
                            elif c == token_size-2:
                                labels[i][a+c+1] = 'E'
    except Exception as e:
        # pass的位置影响BIO的标签
        pass
    i = i+1

# label "o"
for i in range(len(labels)):
    labels[i] = ['O' if j=='' else j for j in labels[i]]



PICO4VR一体机【窦靖童代言】8+256G年度旗舰爆款新机正式发售智能眼镜VR眼镜
[['一', '体', '机']]


In [12]:
# label "sep"
for i in range(len(labels)):
    labels[i].insert(len(labels[i]),'')

In [13]:
#示例
print(labels[90])
#print(X_train[1])
print(y_train[90])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B', 'I', 'E', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '']
一体机


## Format dataset to train data set 

In [14]:
formatted = pd.DataFrame(columns={'char', 'cat'})
i=0
for X_t in X_train:
    #print(X_t)
    sentence = X_t
    separated_sen = list(str(sentence).encode("utf-8").decode("utf-8"))
    separated_sen.insert(len(separated_sen),'')
    sen_labels = list(labels[i])
    for char, label in zip(separated_sen, sen_labels):
        data = [{'char': char,'cat': label}]
        # print(data)
        formatted = formatted.append(data,sort=False)
    i=i+1

In [15]:
formatted.head(100)

,cat,char
0,O,联
0,O,想
0,O,拯
0,O,救
0,O,者
0,O,Y
0,O,9
0,O,0
0,O,0
0,O,0


In [17]:
# check the length
len(X_train)


2516

In [18]:
# check the length
len(formatted)

105564

In [19]:
# save to csv
formatted.to_csv('train_202210.csv', encoding='utf-8-sig')

# CSV to txt 

In [11]:
# read the data 
pd.set_option('display.max_rows', None)
train = pd.read_csv('data/train_202210.csv')
train = train[['char','cat']]

In [12]:
# preprocess the data
train['char'] = train['char'].astype(str)
train['cat'] = train['cat'].astype(str)

In [173]:
#train.dropna(subset = ["cat"], inplace=True)
#train.dropna(subset= ["char"], inplace=True)

In [7]:
for i in range(len(train)):
    train['char'][i] = ['' if cha == 'nan' else cha for cha in train['char'][i]]
  

In [13]:
# replace nan
for index, row in train.iterrows():
    if row['char'] == 'nan':
        train.at[index, 'cat'] = ''
        train.at[index, 'char'] = ''

In [14]:
train.head(100)

,char,cat
0,联,O
1,想,O
2,拯,O
3,救,O
4,者,O
5,Y,O
6,9,O
7,0,O
8,0,O
9,0,O


In [24]:
train[train['char'] == '']

,char,cat
109,,
219,,
329,,
390,,
447,,
475,,
509,,
526,,
584,,
653,,


In [25]:
train.to_csv('train_202210.txt', encoding='utf-8-sig',sep=" " ,index=False, header=False)
